In [1]:
import os
import sys
import re

src_path = os.path.abspath(os.path.join(os.getcwd(), "../.."))
util_path = os.path.abspath(os.path.join(src_path, f'Util'))

print(util_path)
if src_path not in sys.path or util_path not in sys.path:
    sys.path.append(src_path)
    sys.path.append(util_path)
DATA_BASEPATH = os.path.abspath(os.path.join(os.getcwd(), "..","../../Data"))

/sfs/gpfs/tardis/home/ujx4ab/ondemand/CBM_Final_Project/src/Util


In [2]:
from pathlib import Path
import json
import numpy as np  
import copy

from Util.data_utils import load_reddit_posts, get_comments_for_post, get_user_comment_history, get_comments_in_post_thread, upload_json_to_file
from LLM.Mistral.minstral_utils import Mistral, MistralPrompt
from Util.posttree import PostTree, UserCommentHistories

/home/ujx4ab/ondemand/CBM_Final_Project/.venv/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# change these 
TEST_TYPES = ['zero_shot'] #['few_shot'] # few_shot, zero_shot, instruct
POST_IDS = ['18w4y9z', '1atcjtj', '1au5njg']

# don't change these
POST_PATH = os.path.abspath(os.path.join(DATA_BASEPATH, 'arcticshift/processed/r_books_posts.csv'))
COMMENT_PATH = os.path.abspath(os.path.join(DATA_BASEPATH, 'arcticshift/cleaned/r_books_comments_cleaned_and_pruned.csv'))
USER_HIST_PATH = os.path.abspath(os.path.join(DATA_BASEPATH, f'arcticshift/cleaned/r_books_comments_cleaned.csv'))
MODEL_NAME = 'mistralai/Mistral-7B-Instruct-v0.3'

In [4]:
for POST_ID in POST_IDS:
    for TEST_TYPE in TEST_TYPES:
        POST_FOLDER_PATH = os.path.abspath(os.path.join(DATA_BASEPATH, f'eval_results/{TEST_TYPE}/{POST_ID}'))
        mistral = Mistral(model_name=MODEL_NAME)
        post_tree = PostTree(
            POST_PATH,
            POST_ID
            )
        post_tree.create_comment_tree(COMMENT_PATH)

        os.makedirs(POST_FOLDER_PATH, exist_ok=True)
        prompt_response_file = os.path.join(POST_FOLDER_PATH, "prompt_response.txt")

        new_tree = copy.deepcopy(post_tree)

        user_histories = UserCommentHistories(
            USER_HIST_PATH,
            POST_ID
            )

        list_lengths = [len(lst) for lst in user_histories.user_histories.values()]

        # just for some ideas of stats
        mean_length = np.mean(list_lengths)
        std_length = np.std(list_lengths)
        print(f"Mean length: {mean_length}")
        print(f"Standard deviation: {std_length}")

        mistral.process_post_tree(post_tree, user_histories, data_basepath=DATA_BASEPATH, test_type=TEST_TYPE)


`low_cpu_mem_usage` was None, now default to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 3/3 [00:17<00:00,  5.67s/it]


Mean length: 19.08298755186722
Standard deviation: 201.5302077812057


Generating responses:   0%|          | 0/257 [00:00<?, ?node/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/home/ujx4ab/ondemand/CBM_Final_Project/.venv/lib64/python3.11/site-packages/transformers/generation/utils.py:2097: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
Generating responses:   2%|▏         | 4/257 [00:59<59:18, 14.06s/node]  Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
